# Ingress

Hasta el momento tenemos los servicios expuestos de la siguiente manera

![title](images/ingress-nodeport)

![title](images/ingress03)

Pero tal vez necesitemos exponerlos de manera privada dentro del clúster o directamente en internet. Para exponerlo en internet hay varias maneras, se puede hacer exponiendo un puerto en las máquinas del clúster, exponiendo un loadBalancer, pero una manera mas sencilla es redireccionar todo el tráfico a través de un ingress controllers.

![title](images/ingress-loadbalancer)

![title](images/ingress-00)

![title](images/ingress04)

![title](images/ingress2-00)

### En minikube

In [7]:
minikube addons enable ingress  

🔎  Verifying ingress addon...
🌟  The 'ingress' addon is enabled


In [156]:
cat yaml/ingress/deployment_microbot.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: microbot
spec:
  replicas: 3
  selector:
    matchLabels:
      role: microbot
  template:
    metadata:
      labels:
        role: microbot
    spec:
      containers:
      - name: microbot
        image: https://hub.docker.com/r/fr3nd/xeyes/
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 8080



In [213]:
kubectl apply -f yaml/ingress/deployment_microbot.yaml

deployment.apps/microbot configured


In [158]:
cat yaml/ingress/service_microbot.yaml

apiVersion: v1
kind: Service
metadata:
  name: microbot
spec:
  type: NodePort
  ports:
  - port: 8080
    targetPort: 8080
    nodePort: 30003
  selector:
    role: microbot


In [71]:
kubectl apply -f yaml/ingress/service_microbot.yaml

service/microbot unchanged


In [102]:
kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/hello-6c7b794697-82ccg      1/1     Running   1          13h
pod/hello-6c7b794697-ckbt5      1/1     Running   1          13h
pod/hello-6c7b794697-lz6hg      1/1     Running   1          13h
pod/microbot-744f5c9c45-959lz   1/1     Running   0          112m
pod/microbot-744f5c9c45-b8552   1/1     Running   0          112m
pod/microbot-744f5c9c45-spvgl   1/1     Running   0          113m

NAME                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE
service/hello        NodePort    10.100.218.209   <none>        8080:30001/TCP   14h
service/kubernetes   ClusterIP   10.96.0.1        <none>        443/TCP          18h
service/microbot     NodePort    10.106.77.191    <none>        8080:30003/TCP   120m

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello      3/3     3            3           18h
deployment.apps/microbot   3/3     3            3           122m

NAME 

In [74]:
minikube service microbot 

|-----------|----------|-------------|-----------------------------|
| NAMESPACE |   NAME   | TARGET PORT |             URL             |
|-----------|----------|-------------|-----------------------------|
| default   | microbot |        8080 | http://192.168.39.143:30003 |
|-----------|----------|-------------|-----------------------------|
🎉  Opening service default/microbot in default browser...


#### Ahora crearemos el ingress

In [120]:
cat yaml/ingress/ingress.yaml

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$1
spec:
  rules:
    - host: hello-world.info
      http:
        paths:
          - path: /
            pathType: Prefix
            backend:
              service:
                name: hello
                port:
                  number: 8080
          - path: /fun
            pathType: Prefix
            backend:
              service:
                name: microbot
                port:
                  number: 8080


In [121]:
kubectl apply -f yaml/ingress/ingress.yaml

ingress.networking.k8s.io/example-ingress configured


In [122]:
kubectl get services

NAME         TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE
hello        NodePort    10.100.218.209   <none>        8080:30001/TCP   14h
kubernetes   ClusterIP   10.96.0.1        <none>        443/TCP          18h
microbot     NodePort    10.106.77.191    <none>        8080:30003/TCP   132m


In [123]:
kubectl get ingress

NAME              CLASS    HOSTS              ADDRESS          PORTS   AGE
example-ingress   <none>   hello-world.info   192.168.39.143   80      28m


In [127]:
kubectl describe ingress example-ingress

Name:             example-ingress
Namespace:        default
Address:          192.168.39.143
Default backend:  default-http-backend:80 (<error: endpoints "default-http-backend" not found>)
Rules:
  Host              Path  Backends
  ----              ----  --------
  hello-world.info  
                    /      hello:8080      172.17.0.5:8080,172.17.0.6:8080,172.17.0.7:8080)
                    /fun   microbot:8080   172.17.0.13:8080,172.17.0.14:8080,172.17.0.15:8080)
Annotations:        nginx.ingress.kubernetes.io/rewrite-target: /$1
Events:
  Type    Reason  Age                  From                      Message
  ----    ------  ----                 ----                      -------
  Normal  CREATE  30m                  nginx-ingress-controller  Ingress default/example-ingress
  Normal  UPDATE  2m52s (x4 over 30m)  nginx-ingress-controller  Ingress default/example-ingress


In [104]:
minikube ip

192.168.39.143


Modificar el archivo de hosts para añadir el alias a la ip de minikube  
```sudo nano /etc/hosts```

añadir la linea:  
```192.168.39.143 hello-world.info``` 

In [217]:
curl hello-world.info/fun
curl hello-world.info/hello

Hello, world!
Version: 2.0.0
Hostname: microbot-744f5c9c45-spvgl
Hello, world!
Version: 2.0.0
Hostname: hello-6c7b794697-lz6hg
